In [ ]:
# https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection/home
import json
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 32
trunc_type = "post"
oov_tok = "<OOV>"
training_size = 20000
padding_type = "post"

In [ ]:
with open("./sarcasm.json") as f:
    datastore = json.load(f)

In [ ]:
sentences = []
labels = []
urls = []

In [ ]:
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [ ]:
training_sentences = sentences[0:training_size]
testing_sentences =  sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(loss='binary_crossentropy', optimizer = 'adam', metric=['acc'])

In [ ]:
model.summary()

In [ ]:
num_epochs = 30

training_padded_final = np.array(training_padded)
training_labels_final = np.array(training_labels)
testing_padded_final = np.array(testing_padded)
testing_labels_final = np.array(testing_labels)



history = model.fit(training_padded_final, 
                    training_labels_final, 
                    epochs = num_epochs, 
                    validation_data=(testing_padded_final, testing_labels_final),
                    verbose=2)

In [ ]:
import matplotlib.pyplot as plt

def plot_graph(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()

plot_graph(history, "acc")
plot_graph(history, "loss")